<a href="https://colab.research.google.com/github/franlopez1234/7506R-1C2023-GRUPO17/blob/Tp_1/Tp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np 
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Preparacion del Dataset

In [29]:
#Importamos el dataset
from google.colab import files

load = load = files.upload()

Saving hotels_train.csv to hotels_train (1).csv


##Mostramos las variables cuantitativas

In [98]:
lista_cuantitativas = ['adults','children', 'babies','previous_cancellations','previous_bookings_not_canceled','booking_changes','adr','required_car_parking_spaces','total_of_special_requests']
df_cuantitativas = hotels_train[lista_cuantitativas] #Primero separamos las cuantitvas
df_cuantitativas

,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,adr,required_car_parking_spaces,total_of_special_requests
0,1,0.0,0,0,0,0,115.50,0,1
1,2,0.0,0,0,0,0,42.00,1,0
2,2,2.0,0,0,0,0,229.00,0,2
3,2,0.0,0,0,0,0,75.00,0,0
4,2,0.0,0,0,0,0,75.00,0,1
...,...,...,...,...,...,...,...,...,...
61908,2,0.0,0,0,0,0,48.00,0,1
61909,1,0.0,0,0,0,0,149.00,0,0
61910,1,0.0,0,0,0,0,95.00,0,0
61911,2,0.0,0,0,0,0,36.05,0,0


Mostramos media, moda y mediana

In [58]:
df_cuantitativas.mean()

adults                              1.862791
children                            0.104896
babies                              0.007155
previous_cancellations              0.114790
previous_bookings_not_canceled      0.115420
booking_changes                     0.195952
adr                               102.383773
required_car_parking_spaces         0.049957
total_of_special_requests           0.523703
dtype: float64

In [108]:
df_cuantitativas.mode().squeeze()

adults                             2.0
children                           0.0
babies                             0.0
previous_cancellations             0.0
previous_bookings_not_canceled     0.0
booking_changes                    0.0
adr                               62.0
required_car_parking_spaces        0.0
total_of_special_requests          0.0
Name: 0, dtype: float64

In [64]:
df_cuantitativas.median()

adults                             2.0
children                           0.0
babies                             0.0
previous_cancellations             0.0
previous_bookings_not_canceled     0.0
booking_changes                    0.0
adr                               95.0
required_car_parking_spaces        0.0
total_of_special_requests          0.0
dtype: float64

##Mostramos las variables cualitativas

In [104]:
df_cualitativas = hotels_train.drop(lista_cuantitativas, axis=1)
df_cualitativas

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,meal,country,...,assigned_room_type,deposit_type,agent,company,days_in_waiting_list,customer_type,reservation_status,reservation_status_date,id,is_canceled
0,City Hotel,49,2016,September,37,5,1,2,BB,USA,...,D,No Deposit,9.0,NaN,0,Transient,Canceled,2016-08-25,7aa4cc6b-b92c-4061-b21d-4f9e0cac4689,1
1,Resort Hotel,4,2015,October,44,31,0,1,BB,PRT,...,A,No Deposit,NaN,NaN,0,Transient,Check-Out,2015-11-01,b1428f80-c56c-4ae4-91a9-6962edae08b1,0
2,City Hotel,25,2016,October,42,13,0,1,BB,CHE,...,F,No Deposit,9.0,NaN,0,Transient,Check-Out,2016-10-14,2226c018-ea20-4c4e-8f71-ea66aee70d7c,0
3,City Hotel,26,2016,December,51,12,2,5,BB,PRT,...,A,Non Refund,154.0,NaN,0,Transient,Canceled,2016-11-29,314f56dc-2e4d-4298-9bdd-27bb008bffff,1
4,City Hotel,104,2017,March,12,19,2,5,BB,FRA,...,A,No Deposit,20.0,NaN,0,Transient-Party,Check-Out,2017-03-26,28c75445-c603-413a-babf-ad377018b592,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61908,Resort Hotel,24,2017,January,4,27,0,2,BB,ESP,...,D,No Deposit,240.0,NaN,0,Transient,Canceled,2017-01-23,32bffef5-5404-4e86-9f39-fd2282e1ad7f,1
61909,City Hotel,0,2016,September,36,2,0,1,SC,FRA,...,A,No Deposit,NaN,NaN,0,Transient,Check-Out,2016-09-03,cc922a1c-356e-42a5-a984-0b1af03c2f5c,0
61910,City Hotel,10,2017,May,19,8,1,2,BB,FRA,...,A,No Deposit,NaN,153.0,0,Transient,Check-Out,2017-05-11,76fb0e83-59df-43ee-9c33-aec51030ed45,0
61911,Resort Hotel,143,2015,October,41,10,2,5,BB,PRT,...,A,Non Refund,96.0,NaN,0,Transient,Canceled,2015-07-08,01906859-1094-4e37-b13e-b015b594d159,1


In [135]:
hotels_train.isna().sum()

hotel                                 0
lead_time                             0
arrival_date_year                     0
arrival_date_month                    0
arrival_date_week_number              0
arrival_date_day_of_month             0
stays_in_weekend_nights               0
stays_in_week_nights                  0
adults                                0
children                              4
babies                                0
meal                                  0
country                             221
market_segment                        0
distribution_channel                  0
is_repeated_guest                     0
previous_cancellations                0
previous_bookings_not_canceled        0
reserved_room_type                    0
assigned_room_type                    0
booking_changes                       0
deposit_type                          0
agent                              7890
company                           58761
days_in_waiting_list                  0


In [134]:
#df_cualitativas.value_counts().reset_index() buscar forma de encontrar el value counts
#df_cualitativas.value_counts().index

In [ ]:
#Columnas que no sirven: company (llenos nulls) , arrival_date_year , arrival_date_week_number , adr , id 


#Dudoso: hotel , arrival_date_day_of_month  , stays_in_weekend_nights , stays_in_week_nights , reserved_room_type , assigned_room_type , agent


#la usamos: lead_time ,adults -> children,  country, market_segment,distribution_channel , is_repeated_guest, previous_cancellations , previous_bookings_not_canceled , deposit_type , days_in_waiting_list , customer_type , required_car_parking_spaces , reservation_status , reservation_status_date , is_canceled 